In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from sklearn.model_selection import KFold

In [3]:
batch_size = 128
epochs = 1
IMG_HEIGHT = 224
IMG_WIDTH = 224
total_train = 22400
epochs_fine = 10
num_folds = 5
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)
K.clear_session()

In [4]:
# Leemos datos y etiquetas de memoria
x_array = np.load('datos.npy')
y_array = np.load('etiquetas.npy')

In [5]:
# Creamos la clase que nos va a dividir los datos
kfold = KFold(n_splits=num_folds, shuffle=True)

# Variable para almacenar el número de fold por el que vamos
num_fold = 1

# Variables donde almacenar la accuracy y la pérdida de cada fold
accuracies = []
losses = []

# Realizamos la validacion cruzada
for train, test in kfold.split(x_array, y_array): 
    # Obtenemos datos y etiquetas de entrenamiento
    datos_train = x_array[train]
    etiquetas_train = y_array[train]
    
    # Obtenemos datos y etiquetas de test
    datos_test = x_array[test]
    etiquetas_test = y_array[test]

    # Create el modelo base
    base_model = tf.keras.applications.DenseNet201(input_shape=IMG_SHAPE,
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False
    
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    prediction_layer = tf.keras.layers.Dense(28, activation='softmax')

    model = tf.keras.Sequential([
      base_model,
      global_average_layer,
      prediction_layer
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    print('------------------------------------------------------------------')
    print(f'Ejecutando fold {num_fold}')
    
    print(f'Ejecutando transfer learning')
    
    history = model.fit(
        x=datos_train, 
        y=etiquetas_train, 
        batch_size=batch_size,
        epochs=epochs)
    
    base_model.trainable = True

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),  
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,),
        metrics=['accuracy']
    )
    
    print(f'Ejecutando fine tuning')
    
    history = model.fit(
        x=datos_train, 
        y=etiquetas_train, 
        batch_size=batch_size,
        epochs=epochs_fine)

    # Evaluamos el modelo con los datos de test
    scores = model.evaluate(datos_test, etiquetas_test, verbose=0)
    
    print(f'fold={num_fold}: perdida={scores[0]} - accuracy={scores[1]*100}%')
    
    accuracies.append(scores[1] * 100)
    losses.append(scores[0])
    
    K.clear_session()

    # Pasamos el siguiente fold
    num_fold = num_fold + 1

------------------------------------------------------------------
Ejecutando fold 1
Ejecutando transfer learning
175/175 [==============================] - 64s 239ms/step - loss: 2.2395 - accuracy: 0.4197
Ejecutando fine tuning
Epoch 1/10
175/175 [==============================] - 158s 737ms/step - loss: 3.3336 - accuracy: 0.4088
Epoch 2/10
175/175 [==============================] - 122s 697ms/step - loss: 0.4239 - accuracy: 0.8929
Epoch 3/10
175/175 [==============================] - 122s 695ms/step - loss: 0.2214 - accuracy: 0.9406
Epoch 4/10
175/175 [==============================] - 123s 702ms/step - loss: 0.1402 - accuracy: 0.9591
Epoch 5/10
175/175 [==============================] - 121s 691ms/step - loss: 0.1001 - accuracy: 0.9750
Epoch 6/10
175/175 [==============================] - 118s 676ms/step - loss: 0.0692 - accuracy: 0.9825
Epoch 7/10
175/175 [==============================] - 120s 685ms/step - loss: 0.0479 - accuracy: 0.9908
Epoch 8/10
175/175 [=======================

In [6]:
# Mostramos los resultados
for i in range(len(accuracies)):
    print('-------------------------------------------------------------------')
    print(f'Fold {i+1}: Loss={losses[i]} - Accuracy={accuracies[i]}%')
    
print('-------------------------------------------------------------------')
print(f'Final: Loss={np.mean(losses)} - Accuracy={np.mean(accuracies)}% (+- {np.std(accuracies)})')

-------------------------------------------------------------------
Fold 1: Loss=0.09778350591659546 - Accuracy=96.78571224212646%
-------------------------------------------------------------------
Fold 2: Loss=0.08569685369729996 - Accuracy=97.03571200370789%
-------------------------------------------------------------------
Fold 3: Loss=0.08476965129375458 - Accuracy=97.0714271068573%
-------------------------------------------------------------------
Fold 4: Loss=0.09240531921386719 - Accuracy=97.17857241630554%
-------------------------------------------------------------------
Fold 5: Loss=0.09146340936422348 - Accuracy=97.23165035247803%
-------------------------------------------------------------------
Final: Loss=0.09042374789714813 - Accuracy=97.06061482429504% (+- 0.15458048455358253)


In [ ]:
model_name = 'modeloFinalReal.h5'

In [16]:
model.save(model_name)

In [6]:
new_model = tf.keras.models.load_model(model_name)

In [18]:
pred = new_model.predict(x_array)

In [27]:
pred[0]

array([3.52496536e-05, 4.26581028e-05, 4.83288964e-10, 1.47379169e-05,
       2.89587064e-07, 1.05237192e-03, 3.27298039e-04, 1.22638352e-01,
       2.08580673e-06, 4.47910224e-06, 8.05806984e-12, 1.22808362e-03,
       8.22458730e-07, 1.09876972e-04, 8.63652458e-05, 8.70499253e-01,
       4.27346735e-04, 1.09955067e-06, 4.80066582e-08, 1.93857122e-05,
       1.19256205e-08, 5.21227194e-04, 2.02319073e-03, 6.92204367e-06,
       1.17451680e-04, 6.79058721e-04, 1.11104591e-05, 1.51252694e-04],
      dtype=float32)

In [36]:
num = 5
np.argmax(pred[num]) == y_array[num]

True

In [20]:
new_scores = new_model.evaluate(datos_test, etiquetas_test, verbose=0)
print(f'Score: {new_model.metrics_names[0]} of {new_scores[0]}; {new_model.metrics_names[1]} of {new_scores[1]*100}%')

Score: loss of 0.09432641416788101; accuracy of 96.910160779953%
